#models
dosyasına agırlık modelini ekleyin

In [1]:
import os
print(os.getcwd())

models_dir = 'models'
if not os.path.exists(models_dir):
    os.makedirs(models_dir)
    print(f"{models_dir} klasörü oluşturuldu.")
else:
    print(f"{models_dir} klasörü zaten var.")

!pip install pygame numpy torch torchvision matplotlib pandas

/content
models klasörü oluşturuldu.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 839.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      S

#Scriptler'i Olustur

In [13]:
%%writefile constants.py
# Oyun sabitleri
CELL_SIZE = 20
GRID_SIZE = 32
HEADER_HEIGHT = 60
SCREEN_WIDTH = CELL_SIZE * GRID_SIZE
SCREEN_HEIGHT = CELL_SIZE * GRID_SIZE + HEADER_HEIGHT
INITIAL_SPEED = 5  # Başlangıç hızı
SPEED_INCREASE = 1  # Hız artış miktarı
MAX_SPEED = 15  # Maksimum hız sınırı

# Renkler
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
GREEN = (50, 205, 50)
RED = (255, 0, 0)
GRID_COLOR = (40, 40, 40)
BUTTON_COLOR = (70, 130, 180)  # Steel Blue
BUTTON_HOVER_COLOR = (100, 149, 237)  # Cornflower Blue
HEADER_COLOR = (25, 25, 25)  # Koyu gri

Writing constants.py


In [14]:
%%writefile direction.py
from enum import Enum

class Direction(Enum):
    UP = 1
    DOWN = 2
    LEFT = 3
    RIGHT = 4

Writing direction.py


In [15]:
%%writefile ai_model.py
# Gerekli kütüphanelerin import edilmesi
import torch  # PyTorch - derin öğrenme framework'ü
import torch.nn as nn  # Sinir ağı modülleri
import torch.optim as optim  # Optimizasyon algoritmaları
import numpy as np  # Numerik işlemler için
import random  # Rastgele sayı üretimi için
from collections import deque  # Deneyim hafızası için çift yönlü kuyruk
from direction import Direction  # Yön enumları
from constants import GRID_SIZE  # Oyun sabitleri
import os
from google.colab import files

class DQN(nn.Module):
    """Deep Q-Network (DQN) sınıfı

    Bu sınıf, derin Q-öğrenme için kullanılan sinir ağı modelini tanımlar.
    3 katmanlı bir yapıya sahiptir: giriş katmanı -> gizli katman -> çıkış katmanı
    """
    def __init__(self, input_size, hidden_size, output_size):
        """DQN modelinin yapılandırılması

        Args:
            input_size (int): Giriş katmanı boyutu (durum vektörü boyutu)
            hidden_size (int): Gizli katman boyutu
            output_size (int): Çıkış katmanı boyutu (aksiyon sayısı)
        """
        super(DQN, self).__init__()
        # Sinir ağı mimarisi: Giriş -> ReLU -> Gizli -> ReLU -> Çıkış
        self.network = nn.Sequential(
            nn.Linear(input_size, hidden_size),  # Giriş -> Gizli katman
            nn.ReLU(),  # Aktivasyon fonksiyonu
            nn.Linear(hidden_size, hidden_size),  # Gizli -> Gizli katman
            nn.ReLU(),  # Aktivasyon fonksiyonu
            nn.Linear(hidden_size, output_size)  # Gizli -> Çıkış katmanı
        )

    def forward(self, x):
        """İleri yayılım işlemi

        Args:
            x (torch.Tensor): Giriş tensörü (durum)

        Returns:
            torch.Tensor: Q-değerleri tensörü
        """
        return self.network(x)

class SnakeAI:
    """Yılan AI sınıfı

    Bu sınıf, yılanın davranışlarını kontrol eden yapay zeka ajanını temsil eder.
    DQN algoritması kullanarak yılanın optimal hareketleri öğrenmesini sağlar.
    """
    def __init__(self, state_size=21, hidden_size=256, action_size=3):
        """SnakeAI sınıfının başlatılması

        Args:
            state_size (int): Durum vektörünün boyutu (default: 21)
            hidden_size (int): Gizli katman boyutu (default: 256)
            action_size (int): Aksiyon uzayı boyutu (default: 3)
        """
        # Temel parametreler
        self.state_size = state_size  # Durum vektörü boyutu
        self.action_size = action_size  # Aksiyon sayısı
        self.memory = deque(maxlen=100000)  # Deneyim hafızası (son 100,000 deneyim)

        # Öğrenme parametreleri
        self.gamma = 0.98  # İndirim faktörü (gelecek ödüllerin ağırlığı)
        self.epsilon = 1.0  # Başlangıç keşif oranı
        self.epsilon_min = 0.02  # Minimum keşif oranı
        self.epsilon_decay = 0.998  # Keşif oranı azalma katsayısı
        self.learning_rate = 0.0005  # Öğrenme oranı

        # Cihaz seçimi (GPU varsa GPU, yoksa CPU)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Ana model ve hedef model oluşturma
        self.model = DQN(state_size, hidden_size, action_size).to(self.device)
        self.target_model = DQN(state_size, hidden_size, action_size).to(self.device)

        # Optimizer tanımlama (Adam optimizer)
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)

        # Hedef modeli ana model ile senkronize et
        self.update_target_model()

    def update_target_model(self):
        self.target_model.load_state_dict(self.model.state_dict())

    def get_state(self, game):
        """Oyunun mevcut durumunu bir durum vektörüne dönüştürür

        Bu fonksiyon, oyunun mevcut durumunu AI'nın anlayabileceği bir formata çevirir.
        Toplam 21 özellikten oluşan bir durum vektörü oluşturur.

        Args:
            game: Oyun nesnesi

        Returns:
            numpy.array: 21 elemanlı durum vektörü
        """
        # Yılanın başı ve etrafındaki noktalar
        head = game.snake[0]  # Yılanın başı
        point_l = (head[0] - 1, head[1])  # Sol nokta
        point_r = (head[0] + 1, head[1])  # Sağ nokta
        point_u = (head[0], head[1] - 1)  # Üst nokta
        point_d = (head[0], head[1] + 1)  # Alt nokta

        # Yılanın mevcut yönü
        dir_l = game.direction == Direction.LEFT
        dir_r = game.direction == Direction.RIGHT
        dir_u = game.direction == Direction.UP
        dir_d = game.direction == Direction.DOWN

        # Durum vektörü (toplam 21 özellik):
        state = [
            # 1-3: Tehlike algılama (3 özellik)
            # Önde tehlike var mı?
            int((dir_r and self.is_collision(game, point_r)) or
                (dir_l and self.is_collision(game, point_l)) or
                (dir_u and self.is_collision(game, point_u)) or
                (dir_d and self.is_collision(game, point_d))),

            # Sağda tehlike var mı?
            int((dir_u and self.is_collision(game, point_r)) or
                (dir_d and self.is_collision(game, point_l)) or
                (dir_l and self.is_collision(game, point_u)) or
                (dir_r and self.is_collision(game, point_d))),

            # Solda tehlike var mı?
            int((dir_d and self.is_collision(game, point_r)) or
                (dir_u and self.is_collision(game, point_l)) or
                (dir_r and self.is_collision(game, point_u)) or
                (dir_l and self.is_collision(game, point_d))),

            # 4-7: Hareket yönü (4 özellik)
            int(dir_l),  # Sola mı gidiyor?
            int(dir_r),  # Sağa mı gidiyor?
            int(dir_u),  # Yukarı mı gidiyor?
            int(dir_d),  # Aşağı mı gidiyor?

            # 8-11: Elma konumu (4 özellik)
            int(game.apple[0] < head[0]),  # Elma solda mı?
            int(game.apple[0] > head[0]),  # Elma sağda mı?
            int(game.apple[1] < head[1]),  # Elma yukarıda mı?
            int(game.apple[1] > head[1]),  # Elma aşağıda mı?

            # 12-13: Elma mesafesi (2 özellik)
            abs(game.apple[0] - head[0]) / GRID_SIZE,  # X ekseni mesafesi (normalize edilmiş)
            abs(game.apple[1] - head[1]) / GRID_SIZE,  # Y ekseni mesafesi (normalize edilmiş)

            # 14: Yılanın mevcut yönü (1 özellik)
            int(game.direction.value) / 4.0,  # Normalize edilmiş yön değeri

            # 15-20: Yılan vücut bilgileri (6 özellik)
            len(game.snake) / GRID_SIZE,  # Normalize edilmiş yılan uzunluğu

            # Vücut parçaları var mı? (4 yön)
            int(point_l in game.snake[1:]),  # Solda vücut var mı?
            int(point_r in game.snake[1:]),  # Sağda vücut var mı?
            int(point_u in game.snake[1:]),  # Yukarıda vücut var mı?
            int(point_d in game.snake[1:]),  # Aşağıda vücut var mı?

            # 21: Kuyruk yönü (2 özellik)
            int(game.snake[-1][0] < head[0]) - int(game.snake[-1][0] > head[0]),  # X ekseni kuyruk yönü
            int(game.snake[-1][1] < head[1]) - int(game.snake[-1][1] > head[1])   # Y ekseni kuyruk yönü
        ]

        return np.array(state, dtype=np.float32)

    def is_collision(self, game, point):
        """Verilen noktada çarpışma olup olmadığını kontrol eder

        Args:
            game: Oyun nesnesi
            point (tuple): Kontrol edilecek nokta (x, y)

        Returns:
            bool: Çarpışma varsa True, yoksa False
        """
        # Duvarlarla çarpışma kontrolü
        if point[0] < 0 or point[0] >= GRID_SIZE or \
           point[1] < 0 or point[1] >= GRID_SIZE:
            return True
        # Yılanın kendisiyle çarpışma kontrolü (baş hariç tüm vücut)
        if point in game.snake[1:]:
            return True
        return False

    def remember(self, state, action, reward, next_state, done):
        """Deneyimi hafızaya kaydeder (Experience Replay)

        Args:
            state: Mevcut durum
            action: Seçilen aksiyon
            reward: Alınan ödül
            next_state: Sonraki durum
            done: Oyun bitti mi?
        """
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        """Verilen duruma göre bir aksiyon seçer

        Epsilon-greedy stratejisi kullanır:
        - epsilon olasılıkla rastgele aksiyon (keşif)
        - 1-epsilon olasılıkla en iyi aksiyon (kullanım)

        Args:
            state: Mevcut durum

        Returns:
            int: Seçilen aksiyon indeksi
        """
        # Epsilon olasılığıyla rastgele aksiyon seç (keşif)
        if random.random() <= self.epsilon:
            return random.randrange(self.action_size)

        # En yüksek Q-değerine sahip aksiyonu seç (kullanım)
        state = torch.FloatTensor(state).unsqueeze(0).to(self.device)
        with torch.no_grad():  # Gradient hesaplama
            act_values = self.model(state)
        return torch.argmax(act_values).item()

    def replay(self, batch_size):
        """Deneyim tekrarı ile öğrenme gerçekleştirir

        Args:
            batch_size (int): Mini-batch boyutu

        Returns:
            float: Kayıp değeri veya None (yeterli deneyim yoksa)
        """
        # Yeterli deneyim yoksa öğrenme yapma
        if len(self.memory) < batch_size:
            return None

        # Rastgele deneyim örnekleri seç ve numpy dizilerine dönüştür
        minibatch = random.sample(self.memory, batch_size)
        states = np.array([i[0] for i in minibatch])
        actions = np.array([i[1] for i in minibatch])
        rewards = np.array([i[2] for i in minibatch])
        next_states = np.array([i[3] for i in minibatch])
        dones = np.array([i[4] for i in minibatch])

        # Numpy dizilerini PyTorch tensörlerine çevir
        states = torch.FloatTensor(states).to(self.device)
        actions = torch.LongTensor(actions).to(self.device)
        rewards = torch.FloatTensor(rewards).to(self.device)
        next_states = torch.FloatTensor(next_states).to(self.device)
        dones = torch.FloatTensor(dones).to(self.device)

        # Bellman denklemi ile Q-değerlerini güncelle
        # 1. Mevcut durumlar için Q-değerleri
        current_q = self.model(states).gather(1, actions.unsqueeze(1))

        # 2. Sonraki durumlar için maksimum Q-değerleri (hedef ağ kullanarak)
        with torch.no_grad():
            next_q = self.target_model(next_states).max(1)[0]
        # 3. Hedef Q-değerleri = ödül + (1-done) * gamma * max_next_q
        target_q = rewards + (1 - dones) * self.gamma * next_q

        # Kayıp hesaplama (MSE) ve geri yayılım
        loss = nn.MSELoss()(current_q.squeeze(), target_q)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # Epsilon değerini azalt (keşif oranını düşür)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

        return loss.item()

    def load(self, name):
        """Eğitilmiş modeli dosyadan yükler

        Args:
            name (str): Model dosyasının yolu
        """
        if os.path.exists(name):  # Dosyanın var olup olmadığını kontrol et
            print(f"Loading model from: {name}")
            checkpoint = torch.load(name)  # Kaydedilmiş modeli yükle
            self.model.load_state_dict(checkpoint["model_state_dict"])  # Modeli yükle
            self.target_model.load_state_dict(checkpoint["target_model_state_dict"])  # Hedef modeli yükle
            self.optimizer.load_state_dict(checkpoint["optimizer_state_dict"])  # Optimizerı yükle
            self.epsilon = checkpoint["epsilon"]  # Keşif oranını güncelle
            #self.target_model.load_state_dict(self.model.state_dict())
        else:
            print(f"Model dosyası bulunamadı: {name}. Yeni model başlatılıyor.")


    def save(self, name):
        """Mevcut modeli dosyaya kaydeder

        Args:
            name (str): Kaydedilecek dosyanın yolu
        """

        checkpoint = {
            "model_state_dict": self.model.state_dict(),
            "target_model_state_dict": self.target_model.state_dict(),
            "optimizer_state_dict": self.optimizer.state_dict(),
            "epsilon": self.epsilon  # Keşif oranı da kaydediliyor
        }
        torch.save(checkpoint, name)
        file = os.path.join("/content/", name)  # Daha güvenli yöntem
        #files.download(file)

    def check_epsilon_reset(self):
        """Belirli sayıda oyun sonrası epsilon değerini yeniden ayarlar

        Bu fonksiyon, modelin belirli aralıklarla yeniden keşif yapmasını sağlar.
        Böylece yerel optimumlara takılması engellenir.
        """
        self.current_game += 1
        if self.current_game % self.games_before_reset == 0:
            self.epsilon = max(self.epsilon_reset_value, self.epsilon)
            print(f"\nEpsilon reset edildi: {self.epsilon:.3f}")

Writing ai_model.py


In [16]:
%%writefile game.py
# Gerekli kütüphanelerin import edilmesi
import pygame  # Oyun arayüzü için
import random  # Rastgele sayı üretimi için
import numpy as np  # Numerik işlemler için
from direction import Direction  # Yön enumları
from constants import *  # Oyun sabitleri

# Oyunda kullanılacak renkler (RGB formatında)
BLACK = (0, 0, 0)  # Arka plan rengi
WHITE = (255, 255, 255)  # Yazı ve çerçeve rengi
GREEN = (50, 205, 50)  # Yılan rengi
RED = (255, 0, 0)  # Elma rengi
GRID_COLOR = (40, 40, 40)  # Izgara çizgileri rengi
BUTTON_COLOR = (70, 130, 180)  # Buton normal rengi (Steel Blue)
BUTTON_HOVER_COLOR = (100, 149, 237)  # Buton hover rengi (Cornflower Blue)
HEADER_COLOR = (25, 25, 25)  # Başlık arkaplan rengi (Koyu gri)

# Oyun parametreleri ve sabitleri
CELL_SIZE = 20  # Her hücrenin boyutu (piksel)
GRID_SIZE = 32  # Izgara boyutu (32x32)
HEADER_HEIGHT = 60  # Üst bilgi çubuğu yüksekliği
SCREEN_WIDTH = CELL_SIZE * GRID_SIZE  # Ekran genişliği
SCREEN_HEIGHT = CELL_SIZE * GRID_SIZE + HEADER_HEIGHT  # Ekran yüksekliği
INITIAL_SPEED = 5  # Başlangıç oyun hızı
SPEED_INCREASE = 1  # Her artışta eklenecek hız miktarı
MAX_SPEED = 15  # Maksimum oyun hızı

class Button:
    """Oyun arayüzünde kullanılan butonların sınıfı

    Bu sınıf, oyundaki tüm butonların (hız kontrolü, tekrar oyna vb.)
    görünümünü ve davranışını yönetir.
    """
    def __init__(self, x, y, width, height, text):
        """Buton nesnesinin başlatılması

        Args:
            x (int): Butonun x koordinatı
            y (int): Butonun y koordinatı
            width (int): Buton genişliği
            height (int): Buton yüksekliği
            text (str): Buton üzerindeki yazı
        """
        self.rect = pygame.Rect(x, y, width, height)  # Butonun dikdörtgen alanı
        self.text = text  # Buton yazısı
        self.is_hovered = False  # Fare üzerinde mi?
        self.font = pygame.font.Font(None, 24)  # Yazı tipi ve boyutu

    def draw(self, screen):
        """Butonu ekrana çizer

        Args:
            screen: Pygame ekran nesnesi
        """
        # Buton rengini belirle (fare üzerindeyse farklı renk)
        color = BUTTON_HOVER_COLOR if self.is_hovered else BUTTON_COLOR

        # Butonu çiz
        pygame.draw.rect(screen, color, self.rect)  # Buton arkaplanı
        pygame.draw.rect(screen, WHITE, self.rect, 2)  # Buton çerçevesi

        # Buton yazısını çiz
        text_surface = self.font.render(self.text, True, WHITE)
        text_rect = text_surface.get_rect(center=self.rect.center)
        screen.blit(text_surface, text_rect)

    def handle_event(self, event):
        """Buton olaylarını işler (fare hareketi ve tıklama)

        Args:
            event: Pygame olay nesnesi

        Returns:
            bool: Tıklandıysa True, değilse False
        """
        # Fare hareketi kontrolü
        if event.type == pygame.MOUSEMOTION:
            mouse_pos = pygame.mouse.get_pos()
            self.is_hovered = self.rect.collidepoint(mouse_pos)
        # Fare tıklaması kontrolü (sadece sol tık)
        elif event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
            mouse_pos = pygame.mouse.get_pos()
            if self.rect.collidepoint(mouse_pos):
                return True
        return False

class SnakeGame:
    """Yılan oyununun ana sınıfı

    Bu sınıf, oyunun tüm mekaniğini yönetir:
    - Oyun döngüsü
    - Grafik arayüzü
    - Yılanın hareketi ve kontrolleri
    - Çarpışma tespiti
    - Skor takibi
    - Eğitim modu özellikleri
    """
    def __init__(self):
        """Oyun nesnesinin başlatılması

        Bu fonksiyon:
        1. Pygame'i başlatır
        2. Ekranı ayarlar
        3. Oyun parametrelerini tanımlar
        4. Butonları oluşturur
        5. Oyunu sıfırlar
        """
        # Pygame'i başlat ve temel ayarları yap
        pygame.init()
        self.show_ui = True  # Arayüz gösterme durumu
        self.is_training = False  # Eğitim modu durumu

        # Ekran boyutlarını ayarla
        self.info_panel_width = 300  # Eğitim paneli genişliği
        total_width = SCREEN_WIDTH + (self.info_panel_width if self.is_training else 0)
        self.screen = pygame.display.set_mode((total_width, SCREEN_HEIGHT))
        pygame.display.set_caption('Snake Game with AI')

        # Oyun zamanlayıcısı ve hız ayarları
        self.clock = pygame.time.Clock()
        self.base_speed = INITIAL_SPEED  # Temel oyun hızı
        self.speed = INITIAL_SPEED  # Güncel oyun hızı
        self.training_speed = INITIAL_SPEED  # Eğitim modu hızı
        self.speed_multiplier = 1  # Hız çarpanı

        # Yılanın yön bilgisi
        self.last_direction = Direction.RIGHT  # Başlangıç yönü

        # Arayüz butonlarını oluştur
        # 1. Tekrar oyna butonu
        button_width = 200
        button_height = 50
        button_x = (SCREEN_WIDTH - button_width) // 2
        button_y = (SCREEN_HEIGHT - button_height) // 2 + 50
        self.replay_button = Button(
            button_x, button_y,
            button_width, button_height,
            "Tekrar Oyna"
        )

        # 2. Hız kontrol butonları
        self.speed_buttons = []
        self.speeds = [1, 8, 16, 128, 512]  # Hız seçenekleri
        button_width = 45
        button_height = 25
        spacing = 8  # Butonlar arası boşluk

        # Butonların toplam genişliğini hesapla
        total_width = len(self.speeds) * (button_width + spacing) - spacing
        start_x = SCREEN_WIDTH + (self.info_panel_width - total_width) // 2

        # Hız butonlarını oluştur
        for i, speed in enumerate(self.speeds):
            x = start_x + i * (button_width + spacing)
            y = SCREEN_HEIGHT - 50
            self.speed_buttons.append(
                Button(x, y, button_width, button_height, f"{speed}x")
            )

        # Oyunu başlangıç durumuna getir
        self.reset_game()

    def enable_ui(self):
        """Oyun arayüzünü etkinleştirir

        Bu fonksiyon:
        1. Arayüz gösterme bayrağını aktif eder
        2. Eğer Pygame başlatılmamışsa başlatır
        3. Ekranı yeniden ayarlar
        """
        self.show_ui = True  # Arayüzü aktif et

        # Pygame başlatılmamışsa başlat
        if not pygame.display.get_init():
            pygame.init()
            # Eğitim paneli ile birlikte ekranı oluştur
            self.screen = pygame.display.set_mode(
                (SCREEN_WIDTH + self.info_panel_width, SCREEN_HEIGHT)
            )

    def disable_ui(self):
        """Oyun arayüzünü devre dışı bırakır

        Bu fonksiyon:
        1. Arayüz gösterme bayrağını devre dışı bırakır
        2. Eğer Pygame çalışıyorsa kapatır

        Not: Eğitim sırasında performansı artırmak için kullanılır
        """
        self.show_ui = False  # Arayüzü devre dışı bırak

        # Pygame çalışıyorsa kapat
        if pygame.display.get_init():
            pygame.display.quit()

    def reset_game(self):
        """Oyunu başlangıç durumuna getirir

        Bu fonksiyon:
        1. Yılanı başlangıç konumuna yerleştirir
        2. Yeni bir elma yerleştirir
        3. Skoru sıfırlar
        4. Hızı ayarlar
        5. Oyun durumunu sıfırlar
        """
        # Yılanın başlangıç yönü ve konumu
        self.direction = Direction.RIGHT
        self.snake = [
            (GRID_SIZE//2, GRID_SIZE//2),      # Baş
            (GRID_SIZE//2 - 1, GRID_SIZE//2)   # Gövde
        ]

        # Yeni elma yerleştir ve skoru sıfırla
        self.place_apple()
        self.score = 0

        # Eğitim modunda değilse hızı sıfırla
        if not self.is_training:
            self.speed = self.base_speed

        # Oyun durumunu sıfırla
        self.game_over = False
        self.death_cause = None  # Ölüm nedenini sıfırla

    def place_apple(self):
        """Oyun alanına rastgele bir konumda elma yerleştirir

        Not: Elma yılanın üzerine gelmeyecek şekilde yerleştirilir
        """
        while True:
            # Rastgele bir konum seç
            self.apple = (
                random.randint(0, GRID_SIZE-1),  # x koordinatı
                random.randint(0, GRID_SIZE-1)   # y koordinatı
            )
            # Eğer elma yılanın üzerinde değilse döngüyü bitir
            if self.apple not in self.snake:
                break

    def set_training_mode(self, is_training):
        """Eğitim modunu ayarlar

        Eğitim modunda:
        1. Hız çarpanına göre hız ayarlanır
        2. Sağ panelde eğitim bilgileri gösterilir

        Args:
            is_training (bool): Eğitim modu aktif/pasif
        """
        self.is_training = is_training  # Eğitim modunu ayarla

        if is_training:
            # Eğitim hızını ayarla
            self.training_speed = self.base_speed * self.speed_multiplier
            # Eğitim panelini ekle
            self.screen = pygame.display.set_mode(
                (SCREEN_WIDTH + self.info_panel_width, SCREEN_HEIGHT)
            )
        else:
            # Normal moda geç
            self.speed = self.base_speed
            # Eğitim panelini kaldır
            self.screen = pygame.display.set_mode(
                (SCREEN_WIDTH, SCREEN_HEIGHT)
            )

    def update_speed(self, multiplier):
        old_multiplier = self.speed_multiplier
        self.speed_multiplier = multiplier
        if self.is_training:
            self.training_speed = self.base_speed * multiplier
            print(f"Eğitim hızı güncellendi: {old_multiplier}x -> {multiplier}x (FPS: {self.training_speed})")

    def handle_input(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return False
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    return False
                if not self.game_over:
                    # Yılanın anlık yönünü kontrol et ve son yönünü kullan
                    if event.key == pygame.K_UP and self.last_direction != Direction.DOWN:
                        self.direction = Direction.UP
                    if event.key == pygame.K_DOWN and self.last_direction != Direction.UP:
                        self.direction = Direction.DOWN
                    if event.key == pygame.K_LEFT and self.last_direction != Direction.RIGHT:
                        self.direction = Direction.LEFT
                    if event.key == pygame.K_RIGHT and self.last_direction != Direction.LEFT:
                        self.direction = Direction.RIGHT

            # Oyun bitti ekranında buton kontrolü
            if self.game_over:
                if self.replay_button.handle_event(event):
                    self.reset_game()

            # Hız kontrol butonları kontrolü
            if self.is_training:  # Sadece eğitim modunda hız kontrolü aktif
                for i, button in enumerate(self.speed_buttons):
                    if button.handle_event(event):
                        self.update_speed(self.speeds[i])

        return True

    def move_snake(self):
        if self.game_over:
            return None

        head = self.snake[0]
        # Yılanın son yönünü kaydet
        self.last_direction = self.direction

        if self.direction == Direction.UP:
            new_head = (head[0], head[1] - 1)
        elif self.direction == Direction.DOWN:
            new_head = (head[0], head[1] + 1)
        elif self.direction == Direction.LEFT:
            new_head = (head[0] - 1, head[1])
        else:
            new_head = (head[0] + 1, head[1])

        # Duvar çarpışma kontrolü
        if (new_head[0] < 0 or new_head[0] >= GRID_SIZE or
            new_head[1] < 0 or new_head[1] >= GRID_SIZE):
            self.game_over = True
            self.death_cause = "DUVAR"
            return self.death_cause

        # Kendine çarpma kontrolü
        if new_head in self.snake:
            self.game_over = True
            self.death_cause = "KUYRUK"
            return self.death_cause

        self.snake.insert(0, new_head)

        # Elma yeme kontrolü
        if new_head == self.apple:
            self.score += 1
            if not self.is_training:  # Eğitim modunda değilse hız artışı
                if self.score % 5 == 0:  # Her 5 elmada bir hız artışı
                    # Maksimum hız kontrolü
                    if self.speed < MAX_SPEED:
                        self.speed += SPEED_INCREASE
            self.place_apple()
        else:
            self.snake.pop()

        return None

    def draw(self):
        if not self.show_ui:
            return

        self.screen.fill(BLACK)

        # Header çizimi
        pygame.draw.rect(self.screen, HEADER_COLOR, (0, 0, SCREEN_WIDTH, HEADER_HEIGHT))

        # Skor ve hız gösterimi
        font = pygame.font.Font(None, 36)

        # Skor
        score_text = font.render(f'Skor: {self.score}', True, WHITE)
        score_rect = score_text.get_rect(midleft=(20, HEADER_HEIGHT//2))
        self.screen.blit(score_text, score_rect)

        # Hız (sadece training modunda değilse göster)
        if not self.is_training:
            speed_text = font.render(f'Hız: {self.speed}', True, WHITE)
            speed_rect = speed_text.get_rect(midright=(SCREEN_WIDTH - 20, HEADER_HEIGHT//2))
            self.screen.blit(speed_text, speed_rect)

        # Izgara çizimi
        for x in range(0, SCREEN_WIDTH, CELL_SIZE):
            pygame.draw.line(self.screen, GRID_COLOR,
                           (x, HEADER_HEIGHT), (x, SCREEN_HEIGHT))
        for y in range(HEADER_HEIGHT, SCREEN_HEIGHT, CELL_SIZE):
            pygame.draw.line(self.screen, GRID_COLOR,
                           (0, y), (SCREEN_WIDTH, y))

        # Yılan çizimi
        for segment in self.snake:
            pygame.draw.rect(self.screen, GREEN,
                           (segment[0] * CELL_SIZE,
                            segment[1] * CELL_SIZE + HEADER_HEIGHT,
                            CELL_SIZE-1, CELL_SIZE-1))

        # Elma çizimi
        pygame.draw.rect(self.screen, RED,
                        (self.apple[0] * CELL_SIZE,
                         self.apple[1] * CELL_SIZE + HEADER_HEIGHT,
                         CELL_SIZE-1, CELL_SIZE-1))

        if self.game_over:
            # Oyun bitti yazısı
            font = pygame.font.Font(None, 74)
            text = font.render('Oyun Bitti!', True, WHITE)
            text_rect = text.get_rect(center=(SCREEN_WIDTH/2, SCREEN_HEIGHT/2))
            self.screen.blit(text, text_rect)

            # Tekrar oyna butonu
            self.replay_button.draw(self.screen)

    def draw_training_info(self, game, score, record, mean_score, epsilon, steps, fps):
        if not self.show_ui or not self.is_training:
            return

        # Sağ panel arkaplanı
        panel_x = SCREEN_WIDTH
        pygame.draw.rect(self.screen, HEADER_COLOR,
                        (panel_x, 0, self.info_panel_width, SCREEN_HEIGHT))

        # Başlık
        font_large = pygame.font.Font(None, 36)
        title = font_large.render('Eğitim Bilgileri', True, WHITE)
        title_rect = title.get_rect(centerx=panel_x + self.info_panel_width//2, y=20)
        self.screen.blit(title, title_rect)

        # Alt çizgi
        pygame.draw.line(self.screen, WHITE,
                        (panel_x + 20, 50),
                        (panel_x + self.info_panel_width - 20, 50))

        font = pygame.font.Font(None, 28)
        y = 70
        spacing = 35

        # Temel bilgiler
        infos = [
            ('Oyun', f'{game}'),
            ('Skor', f'{score}'),
            ('Rekor', f'{record}'),
            ('Ort. Skor', f'{mean_score:.2f}'),
            ('Epsilon', f'{epsilon:.3f}'),
            ('Adımlar', f'{steps}'),
            ('FPS', f'{self.training_speed}'),
            ('Yılan Uzunluğu', f'{len(self.snake)}'),
            ('Hız Çarpanı', f'{self.speed_multiplier}x')
        ]

        for label, value in infos:
            text = font.render(f'{label}:', True, WHITE)
            value_text = font.render(str(value), True, WHITE)

            # Label sol tarafta
            self.screen.blit(text, (panel_x + 20, y))
            # Değer sağ tarafta
            value_rect = value_text.get_rect(
                right=panel_x + self.info_panel_width - 20,
                top=y
            )
            self.screen.blit(value_text, value_rect)

            y += spacing

        # Aktif hız butonunu vurgula
        for i, button in enumerate(self.speed_buttons):
            is_active = self.speed_multiplier == self.speeds[i]
            if is_active:
                # Aktif buton için ekstra vurgu
                pygame.draw.rect(self.screen, WHITE, button.rect, 3)
            button.draw(self.screen)

        pygame.display.flip()

    def get_state(self):
        # AI için oyun durumunu döndür
        state = np.zeros((GRID_SIZE, GRID_SIZE), dtype=np.float32)

        # Yılanın konumu
        for x, y in self.snake:
            state[y][x] = 0.5

        # Yılan başı
        head_x, head_y = self.snake[0]
        state[head_y][head_x] = 1

        # Elmanın konumu
        state[self.apple[1]][self.apple[0]] = -1

        return state

    def run(self):
        running = True
        while running:
            running = self.handle_input()
            self.move_snake()
            self.draw()
            self.clock.tick(self.speed)
            pygame.display.flip()  # Ekranı güncelle

import os

# Google Colab'de olup olmadığını kontrol et
def is_running_in_colab():
    print(os.environ)
    return "COLAB_GPU" in os.environ or "COLAB_KERNEL" in os.environ

if not is_running_in_colab():
    if __name__ == "__main__":
        game = SnakeGame()
        game.run()
        pygame.quit()

Writing game.py


In [17]:
%%writefile train.py
# Gerekli kütüphanelerin import edilmesi
import pygame  # Oyun arayüzü için
import numpy as np  # Numerik işlemler için
from game import SnakeGame  # Yılan oyunu sınıfı
from ai_model import SnakeAI  # Yapay zeka modeli
from direction import Direction  # Yön enumları
from constants import INITIAL_SPEED, GRID_SIZE  # Oyun sabitleri
import os  # Dosya işlemleri için
import time  # Zaman ölçümleri için
from collections import deque  # Sabit boyutlu kuyruk yapısı için

class TrainLogger:
    """Eğitim sürecini izlemek ve kayıt tutmak için kullanılan sınıf

    Bu sınıf, eğitim sürecinde elde edilen skor, epsilon, kayıp gibi
    değerleri ve çeşitli istatistikleri tutar ve günceller.
    """
    def __init__(self, log_size=100):
        """TrainLogger sınıfının başlatılması

        Args:
            log_size (int): Skor geçmişi için maksimum kayıt sayısı (default: 100)
        """
        # Temel metrikler
        self.scores = deque(maxlen=log_size)  # Son N oyunun skorları
        self.mean_scores = []  # Ortalama skorların geçmişi
        self.max_score = 0  # En yüksek skor
        self.total_games = 0  # Toplam oyun sayısı
        self.epsilon_history = []  # Epsilon değerlerinin geçmişi
        self.loss_history = []  # Kayıp değerlerinin geçmişi

        # Ölüm ve performans istatistikleri
        self.wall_deaths = 0  # Duvara çarpma sayısı
        self.self_deaths = 0  # Kendine çarpma sayısı
        self.total_apples = 0  # Toplanan toplam elma sayısı
        self.longest_snake = 0  # En uzun yılan uzunluğu

    def update(self, score, epsilon, loss):
        """Eğitim metriklerini günceller

        Args:
            score (int): Mevcut oyunun skoru
            epsilon (float): Güncel epsilon değeri
            loss (float): Güncel kayıp değeri (None olabilir)

        Returns:
            float: Güncel ortalama skor
        """
        # Temel metrikleri güncelle
        self.scores.append(score)  # Yeni skoru ekle
        self.max_score = max(self.max_score, score)  # Rekor güncelleme
        self.total_games += 1  # Oyun sayısını artır
        self.epsilon_history.append(epsilon)  # Epsilon geçmişini güncelle

        # Kayıp değeri varsa kaydet
        if loss is not None:
            self.loss_history.append(loss)

        # Ortalama skoru hesapla ve kaydet
        mean_score = np.mean(list(self.scores))
        self.mean_scores.append(mean_score)

        return mean_score



def train(load_from_checkpoint=False,cevrim=1000,speed =8,render=False):
    """Yılanın eğitim sürecini yöneten ana fonksiyon

    Bu fonksiyon:
    1. Eğitim parametrelerini ayarlar
    2. Oyun ve AI modelini başlatır
    3. Eğitim döngüsünü yönetir
    4. Modeli belirli aralıklarla kaydeder
    5. İstatistikleri tutar ve gösterir
    """
    # Eğitim parametreleri
    n_games = cevrim  # Toplam eğitim oyunu sayısı
    batch_size = 32  # Mini-batch boyutu
    record = 0  # Rekor skor
    t_games=0

    # Gerekli nesneleri oluştur
    ai = SnakeAI()  # Yapay zeka modeli
    game = SnakeGame()  # Oyun motoru
    logger = TrainLogger()  # Eğitim loglayıcı

    # Model kayıt klasörünü oluştur
    if not os.path.exists('models'):
        os.makedirs('models')
    else:
        if load_from_checkpoint:
            checkpoint_files = [f for f in os.listdir("models") if f.endswith('.pth')]

            # Eğer checkpoint dosyası varsa, en son kaydedilen dosyayı yükle
            if checkpoint_files:
                # En son kaydedilen dosyayı seç (dosya adındaki çevrim numarasına göre)
                latest_checkpoint = max(checkpoint_files, key=lambda f: int(f.split('_')[-1].split('.')[0]))
                model_path = os.path.join("models", latest_checkpoint)

                # Çevrim numarasını çıkar (dosya adından)
                cycle_number = int(latest_checkpoint.split('_')[-1].split('.')[0])
                t_games=cycle_number  # Çevrim numarasını döndür
                ai.load(f'models/model_checkpoint_{t_games}.pth')
                print(f"Model yüklendi: {model_path}, Çevrim: {cycle_number}")
            else:
                print("Yeni bir model başlatılıyor.")


    print('Eğitim başlıyor...')

    # Eğitim modunu aktifleştir
    game.set_training_mode(True)
    game.training_speed = INITIAL_SPEED * speed  # Başlangıç eğitim hızı 8x

    for i in range(n_games):
        print('\r ', end='', flush=True)  # Satırı temizle
        # Her 50 oyunda bir UI göster
        show_ui = (i % 50 == 0)
        if not show_ui:
            game.disable_ui()  # UI'ı kapat
        else:
            game.enable_ui()   # UI'ı aç
            print(f'\nOyun: {t_games+i+1} başlıyor...')

        # Oyunu sıfırla
        game.reset_game()
        game_over = False
        steps = 0
        last_score = 0
        start_time = time.time()

        # Başlangıç durumu
        state = ai.get_state(game)

        while not game_over:
            if show_ui:
                # Event handling (sadece UI açıkken)
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        pygame.quit()
                        return
                    # Hız kontrol butonları için event handling
                    if event.type == pygame.MOUSEBUTTONDOWN:
                        for i, button in enumerate(game.speed_buttons):
                            if button.handle_event(event):
                                game.update_speed(game.speeds[i])
                    elif event.type == pygame.MOUSEMOTION:
                        for button in game.speed_buttons:
                            button.handle_event(event)

            # AI'nin hareketi seç
            action = ai.act(state)

            # Hareketi uygula
            reward, game_over, score, death_cause = play_step(game, action)

            # Yeni durumu al
            next_state = ai.get_state(game)

            # Hafızaya kaydet
            ai.remember(state, action, reward, next_state, game_over)

            # Durumu güncelle
            state = next_state

            # Öğrenme
            loss = ai.replay(batch_size)

            steps += 1

            # Her 100 adımda bir target modeli güncelle
            if steps % 100 == 0:
                ai.update_target_model()

            if show_ui and render:
                # Oyunu çiz (sadece UI açıkken)
                game.draw()

                # Loglama bilgilerini çiz
                mean_score = logger.update(score, ai.epsilon, loss)
                game.draw_training_info(
                    game=i + 1,
                    score=score,
                    record=record,
                    mean_score=mean_score,
                    epsilon=ai.epsilon,
                    steps=steps,
                    fps=game.training_speed
                )

                pygame.display.flip()
                game.clock.tick(game.training_speed)

        # İstatistikleri güncelle
        if death_cause == "DUVAR":
            logger.wall_deaths += 1
        elif death_cause == "KUYRUK":
            logger.self_deaths += 1

        logger.total_apples += score
        logger.longest_snake = max(logger.longest_snake, len(game.snake))

        elapse = time.time() - start_time
        print(f'{i}- {elapse:.1f}s - {len(game.snake)}', end='', flush=True)

        # Sadece UI açıkken istatistikleri yazdır
        if show_ui:
            duration = time.time() - start_time
            print(f'Oyun Sonucu:')
            #print(f'Skor: {score}, Epsilon: {ai.epsilon:.3f}, Mean Score: {mean_score:.2f}')
            print(f'Steps: {steps}, Süre: {duration:.1f}s, Hız: {game.speed_multiplier}x')
            print(f'Ölüm Nedeni: {death_cause}, Yılan Uzunluğu: {len(game.snake)}')
            print(f'Toplam İstatistikler:')
            print(f'Duvar: {logger.wall_deaths}, Kuyruk: {logger.self_deaths}')
            print(f'Toplam Elma: {logger.total_apples}, En Uzun Yılan: {logger.longest_snake}')
            print('-' * 80)

        # Rekor kontrolü (sadece UI açıkken bildir)
        if score > record:
            record = score
            #ai.save(f'models/model_record_{record}.pth')
            if show_ui:
                print(f'Yeni Rekor! Skor: {record}')

        # Her 100 oyunda bir model kaydet (sadece UI açıkken bildir)
        if (i + 1) % 100 == 0:
            ai.save(f'models/model_checkpoint_{t_games+i+1}.pth')
            if show_ui:
                print(f'Model kaydedildi: Checkpoint {t_games+i+1}')

def play_step(game, action):
    """Oyunda bir adım ilerlemeyi sağlayan fonksiyon

    Bu fonksiyon:
    1. AI'nin seçtiği aksiyonu yöne çevirir
    2. Yılanı hareket ettirir
    3. Ödül hesaplaması yapar

    Args:
        game (SnakeGame): Oyun nesnesi
        action (int): AI'nin seçtiği aksiyon (0: düz, 1: sağ, 2: sol)

    Returns:
        tuple: (ödül, oyun_bitti_mi, skor, ölüm_nedeni)
    """
    # Ödül başlangıcı
    reward = 0

    # Aksiyonu yöne çevir
    clock_wise = [Direction.RIGHT, Direction.DOWN, Direction.LEFT, Direction.UP]  # Saat yönünde yönler
    idx = clock_wise.index(game.direction)  # Mevcut yönün indeksi

    # Yeni yönü belirle
    if action == 0:  # Düz git
        new_dir = clock_wise[idx]
    elif action == 1:  # Sağa dön
        new_dir = clock_wise[(idx + 1) % 4]
    else:  # Sola dön
        new_dir = clock_wise[(idx - 1) % 4]

    # Yeni yönü uygula
    game.direction = new_dir

    # Hareket öncesi durumu kaydet
    old_score = game.score  # Mevcut skor
    old_distance = ((game.snake[0][0] - game.apple[0])**2 +
                   (game.snake[0][1] - game.apple[1])**2)**0.5  # Elmaya olan mesafe
    old_head = game.snake[0]  # Yılanın başı

    # Yılanı hareket ettir
    death_cause = game.move_snake()

    # Hareket sonrası durumu değerlendir
    if not game.game_over:
        # Yeni baş pozisyonu
        head = game.snake[0]

        # Yeni elmaya olan mesafe
        new_distance = ((head[0] - game.apple[0])**2 +
                       (head[1] - game.apple[1])**2)**0.5

        # Her adım için küçük negatif ödül
        # Bu, yılanın gereksiz dönüşler yapmak yerine
        # elmaya doğru hareket etmesini teşvik eder
        reward = -0.1

        # Elmaya yaklaşma/uzaklaşma kontrolü
        # Elmaya yaklaşma durumunu kontrol et
        if new_distance < old_distance:
            reward = 0  # Elmaya yaklaşıyorsa ceza verme (nötr durum)

        # Yerinde sayma kontrolü
        # Yılanın aynı noktada kalmasını engellemek için
        # bu durumu cezalandırıyoruz
        if (old_head[0], old_head[1]) == (head[0], head[1]):
            reward = -1  # Aynı yerde kalma cezası

    # Ölüm durumu kontrolü
    if game.game_over:
        reward = -10  # Ölüm durumunda büyük ceza
        return reward, True, game.score, death_cause

    # Elma yeme kontrolü
    if game.score > old_score:
        reward = 20  # Elma yeme durumunda büyük ödül

    # Durumun sonuçlarını döndür:
    # - reward: Hesaplanan ödül/ceza değeri
    # - game.game_over: Oyun bitti mi?
    # - game.score: Güncel skor
    # - death_cause: Eğer öldüyse ölüm nedeni
    return reward, game.game_over, game.score, death_cause

if __name__ == '__main__':
    # Program doğrudan çalıştırıldığında eğitimi başlat
    load_from_checkpoint = True  # Burada True veya False belirleyebilirsiniz
    train(load_from_checkpoint,1000,8,False)

Writing train.py


#RUN train

In [ ]:
from train import train
if __name__ == '__main__':
    train(True,1000,128,False)


Loading model from: models/model_checkpoint_17900.pth
Model yüklendi: models/model_checkpoint_17900.pth, Çevrim: 17900
Eğitim başlıyor...
 
Oyun: 17901 başlıyor...
0- 5.6s - 37Oyun Sonucu:
Steps: 869, Süre: 5.6s, Hız: 1x
Ölüm Nedeni: KUYRUK, Yılan Uzunluğu: 37
Toplam İstatistikler:
Duvar: 0, Kuyruk: 1
Toplam Elma: 35, En Uzun Yılan: 37
--------------------------------------------------------------------------------
Yeni Rekor! Skor: 35
 
Oyun: 17951 başlıyor...
50- 5.5s - 34Oyun Sonucu:
Steps: 802, Süre: 5.5s, Hız: 1x
Ölüm Nedeni: DUVAR, Yılan Uzunluğu: 34
Toplam İstatistikler:
Duvar: 11, Kuyruk: 40
Toplam Elma: 1309, En Uzun Yılan: 64
--------------------------------------------------------------------------------
 99- 0.9s - 8

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

 
Oyun: 18001 başlıyor...
100- 8.3s - 61Oyun Sonucu:
Steps: 1423, Süre: 8.3s, Hız: 1x
Ölüm Nedeni: DUVAR, Yılan Uzunluğu: 61
Toplam İstatistikler:
Duvar: 28, Kuyruk: 73
Toplam Elma: 2543, En Uzun Yılan: 64
--------------------------------------------------------------------------------
 
Oyun: 18051 başlıyor...
150- 2.5s - 20Oyun Sonucu:
Steps: 417, Süre: 2.5s, Hız: 1x
Ölüm Nedeni: KUYRUK, Yılan Uzunluğu: 20
Toplam İstatistikler:
Duvar: 42, Kuyruk: 109
Toplam Elma: 3758, En Uzun Yılan: 64
--------------------------------------------------------------------------------
 199- 3.1s - 24

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

 
Oyun: 18101 başlıyor...
200- 4.5s - 34Oyun Sonucu:
Steps: 760, Süre: 4.5s, Hız: 1x
Ölüm Nedeni: KUYRUK, Yılan Uzunluğu: 34
Toplam İstatistikler:
Duvar: 56, Kuyruk: 145
Toplam Elma: 5123, En Uzun Yılan: 64
--------------------------------------------------------------------------------
 